
# Use scikit-learn and AI lifecycle capabilities to predict Boston house prices with `ibm-watson-machine-learning`

This notebook contains steps and code to demonstrate support of AI Lifecycle features in Watson Machine Learning Service. It contains steps and code to work with [ibm-watson-machine-learning](https://pypi.python.org/pypi/ibm-watson-machine-learning) library available in PyPI repository. It also introduces commands for getting model and training data, persisting model, deploying model, scoring it, updating the model and redeploying it.

Some familiarity with Python is helpful. This notebook uses Python 3.7.

## Learning goals

The learning goals of this notebook are:

-  Download an externally trained scikit-learn model with dataset.
-  Persist an external model in Watson Machine Learning repository.
-  Deploy model for online scoring using client library.
-  Score sample records using client library.
-  Update previously persisted model.
-  Redeploy model in-place.
-  Scale deployment.


## Contents

This notebook contains the following parts:

1.	[Setup](#setup)
2.	[Download externally created scikit model and data](#download)
3.	[Persist externally created scikit model](#persistence)
4.	[Deploy and score in a Cloud](#scoring)
5.  [Persist new version of the model](#update_model)
6.  [Redeploy new version of the model](#redeploy)
7.  [Deployment scaling](#scale)
8.  [Clean up](#cleanup)
9.	[Summary and next steps](#summary)

<a id="setup"></a>
## 1. Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Contact with your Cloud Pack for Data administrator and ask him for your account credentials

### Connection to WML

Authenticate the Watson Machine Learning service on IBM Cloud Pack for Data. You need to provide platform `url`, your `username` and `password`.

In [ ]:
username = 'PASTE YOUR USERNAME HERE'
password = 'PASTE YOUR PASSWORD HERE'
url = 'PASTE THE PLATFORM URL HERE'

In [2]:
wml_credentials = {
    "username": username,
    "password": password,
    "url": url,
    "instance_id": 'openshift',
    "version": '4.0'
}

Alternatively you can use `username` and `api_key` to authenticate WML services.

```
wml_credentials = {
    "username": ***,
    "api_key": ***,
    "url": ***,
    "instance_id": 'openshift',
    "version": '4.0'
}

```

### Install and import the `ibm-watson-machine-learning` package
**Note:** `ibm-watson-machine-learning` documentation can be found <a href="http://ibm-wml-api-pyclient.mybluemix.net/" target="_blank" rel="noopener no referrer">here</a>.

In [ ]:
!pip install -U ibm-watson-machine-learning

In [3]:
from ibm_watson_machine_learning import APIClient

client = APIClient(wml_credentials)

### Working with spaces

First of all, you need to create a space that will be used for your work. If you do not have space already created, you can use `{PLATFORM_URL}/ml-runtime/spaces?context=icp4data` to create one.

- Click New Deployment Space
- Create an empty space
- Go to space `Settings` tab
- Copy `space_id` and paste it below

**Tip**: You can also use SDK to prepare the space for your work. More information can be found [here](https://github.com/IBM/watson-machine-learning-samples/blob/master/cpd4.0/notebooks/python_sdk/instance-management/Space%20management.ipynb).

**Action**: Assign space ID below

In [ ]:
space_id = 'PASTE YOUR SPACE ID HERE'

You can use `list` method to print all existing spaces.

In [ ]:
client.spaces.list(limit=10)

To be able to interact with all resources available in Watson Machine Learning, you need to set **space** which you will be using.

In [4]:
client.set.default_space(space_id)

'SUCCESS'

<a id="download"></a>
## 2. Download externally created scikit model and data
In this section, you will download externally created scikit models and data used for training it.

In [5]:
!pip install -U wget

import os
import wget

data_dir = 'BOSTON_HOUSE_PRICES_DATA'
if not os.path.isdir(data_dir):
    os.mkdir(data_dir)

model_path = os.path.join(data_dir, 'boston_regressor_0_23_1.tar.gz')
new_model_path = os.path.join(data_dir, 'boston_regressor_new_0_23_1.tar.gz')
if not os.path.isfile(model_path):
    wget.download("https://github.com/IBM/watson-machine-learning-samples/raw/master/cpd4.0/models/scikit/boston_house_price/model/boston_regressor_0_23_1.tar.gz", out=data_dir)  
if not os.path.isfile(new_model_path):
    wget.download("https://github.com/IBM/watson-machine-learning-samples/raw/master/cpd4.0/models/scikit/boston_house_price/model/boston_regressor_new_0_23_1.tar.gz", out=data_dir)  

In [6]:
from sklearn import datasets
import pandas as pd

boston_data = datasets.load_boston()
boston_df = pd.DataFrame(boston_data.data)
boston_df.columns = boston_data.feature_names
boston_df['PRICE'] = boston_data.target

In [7]:
train_df = boston_df
test_df = boston_df.drop(['PRICE'], axis=1)

<a id="persistence"></a>
## 3. Persist externally created scikit model

In this section, you will learn how to store your model in Watson Machine Learning repository by using the Watson Machine Learning Client.

### 3.1: Publish model

#### Publish model in Watson Machine Learning repository on Cloud.

Define model name, autor name and email.

In [8]:
sofware_spec_uid = client.software_specifications.get_id_by_name("default_py3.7")

In [9]:
metadata = {
            client.repository.ModelMetaNames.NAME: 'boston-house-prices-prediction-model',
            client.repository.ModelMetaNames.TYPE: 'scikit-learn_0.23',
            client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: sofware_spec_uid
}

published_model = client.repository.store_model(
    model=model_path,
    meta_props=metadata,
    training_data=train_df)

### 3.2: Get model details

In [ ]:
import json

published_model_uid = client.repository.get_model_uid(published_model)
model_details = client.repository.get_details(published_model_uid)
print(json.dumps(model_details, indent=2))

### 3.3 Get all models

In [ ]:
models_details = client.repository.list_models(limit=10)

<a id="cleanup"></a>
## 4. Clean up 

If you want to clean up all created assets:
- experiments
- trainings
- pipelines
- model definitions
- models
- functions
- deployments

please follow up this sample [notebook](https://github.com/IBM/watson-machine-learning-samples/blob/master/cpd4.0/notebooks/python_sdk/instance-management/Machine%20Learning%20artifacts%20management.ipynb).

<a id="summary"></a>
## 5. Summary and next steps

 You successfully completed this notebook! You learned how to use scikit-learn machine learning as well as Watson Machine Learning for model creation and deployment. 
 
 Check out our <a href="https://dataplatform.cloud.ibm.com/docs/content/analyze-data/wml-setup.html" target="_blank" rel="noopener noreferrer">Online Documentation</a> for more samples, tutorials, documentation, how-tos, and blog posts. 

### Authors

**Daniel Ryszka**, Software Engineer  
**Jan Sołtysik**, Intern 

Copyright © 2020, 2021 IBM. This notebook and its source code are released under the terms of the MIT License.